# Exploratory data analysis (EDA)

Use this notebook to explore and visualize the data.

In [1]:
import os
import ibis
import ibis.selectors as s

import plotly.express as px

from icarus.config import (
    DATA_DIR,
    RAW_DATA_DIR,
    PENGUINS_TABLE,
    BUY_SELL_TABLE,
    SOCIAL_MEDIA_TABLE,
)
from icarus.catalog import Catalog
from icarus.synthetic_data.investments import fake, seed_table

px.defaults.template = "plotly_dark"
ibis.options.interactive = True

In [3]:
catalog = Catalog()
catalog.list_tables()

['buy_sell.delta', 'social_media.delta', 'penguins.delta']

In [4]:
t = catalog.table(PENGUINS_TABLE)
t

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ extracted_at               ┃ species ┃ island    ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ sex    ┃ year  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ string                     │ string  │ string    │ float64        │ float64       │ int64             │ int64       │ string │ int64 │
├────────────────────────────┼─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼────────┼───────┤
│ 2024-08-25T19:17:49.803712 │ Adelie  │ Torgersen │           39.3 │          20.6 │               190 │        3650 │ male   │  2007 │
│ 2024-08-25T19:17:49.803712 │ Gentoo  │ Biscoe    │           43.3 │          14.0 │               208 │        4575 │ female │  2009 │
│ 2024-08-25T19:17:49.803712 │ Adelie  │ Torgersen │           38.7 │          19.0 │               195 │        3450 │ female │  2007 │
│ 2024-08-25T19:17:49.803712 │ Adelie  │ Dream     │           39.5 │          16.7 │               178 │        3250 │ female │  2007 │
│ 2024-08-25T19:17:49.803712 │ Adelie  │ Dream     │           36.4 │          17.0 │               195 │        3325 │ female │  2007 │
│ 2024-08-25T19:17:49.803712 │ Adelie  │ Torgersen │           34.4 │          18.4 │               184 │        3325 │ female │  2007 │
│ 2024-08-25T19:17:49.803712 │ Gentoo  │ Biscoe    │           52.1 │          17.0 │               230 │        5550 │ male   │  2009 │
│ 2024-08-25T19:17:49.803712 │ Adelie  │ Biscoe    │           38.2 │          18.1 │               185 │        3950 │ male   │  2007 │
│ 2024-08-25T19:17:49.803712 │ Adelie  │ Biscoe    │           35.3 │          18.9 │               187 │        3800 │ female │  2007 │
│ 2024-08-25T19:17:49.803712 │ Adelie  │ Biscoe    │           40.5 │          17.9 │               187 │        3200 │ female │  2007 │
│ …                          │ …       │ …         │              … │             … │                 … │           … │ …      │     … │
└────────────────────────────┴─────────┴───────────┴────────────────┴───────────────┴───────────────────┴─────────────┴────────┴───────┘

In [5]:
t.count()

┌───────────────┐
│ np.int64(344) │
└───────────────┘

In [2]:
f"{10_000_000_000 / (60 * 60 * 24):,.2f}"

'115,740.74'

In [ ]:
lookback = ibis.interval(days=1)
step = ibis.interval(seconds=1)

In [ ]:
t = (
    ibis.range(ibis.now() - lookback, ibis.now(), step=step)
    .unnest()
    .name("timestamp")
    .as_table()
    .mutate(
        index=(ibis.row_number().over(order_by="timestamp")),
        symbol=ibis.random(),
        bid=ibis.random(),
        bid_size=ibis.random(),
        ask=ibis.random(),
        ask_size=ibis.random(),
        **{c: 2 * (ibis.random() - 0.5) for c in ["a", "b", "c"]},
        **{
            c: ibis.null(type=str)
            for c in list(map(chr, range(ord("d"), ord("z") + 1)))
        },
    )
    .relocate("index", "timestamp")
)
t

In [ ]:
import random

symbols = [chr(i) for i in range(ord("A"), ord("Z") + 1)]
symbols = [
    f"{a}{b}{c}{d}" for a in symbols for b in symbols for c in symbols for d in symbols
]
symbols = random.sample(symbols, 500)
symbols[:5]

In [ ]:
symbols_case = ibis.case()
for i, symbol in enumerate(symbols):
    symbols_case = symbols_case.when(
        (i / len(symbols) < ibis._["symbol"])
        & (ibis._["symbol"] < ((i + 1) / len(symbols))),
        symbol,
    )
symbols_case = symbols_case.else_(ibis.null()).end()

In [ ]:
t = t.mutate(
    symbol=symbols_case,
    bid_size=(ibis._["bid_size"] * 1000).cast("int64"),
    ask_size=(ibis._["ask_size"] * 1000).cast("int64"),
)
t

In [ ]:
t

In [ ]:
seed_table

In [ ]:
catalog = Catalog()
catalog.list_tables()

In [ ]:
seed_table

In [ ]:
data_glob = os.path.join(DATA_DIR, RAW_DATA_DIR, BUY_SELL_TABLE, "*.parquet")
bs = ibis.read_parquet(data_glob)
bs

In [ ]:
bs = catalog.table("buy_sell")
bs

In [ ]:
t = bs.group_by("ticker").agg(count=ibis._.count()).order_by(ibis.desc("count"))
t

In [ ]:
px.bar(
    t,
    x="ticker",
    y="count",
    title="Number of trades per symbol",
    color="ticker",
)

In [ ]:
bs.count()

In [ ]:
bs["location"].value_counts()

In [ ]:
f"{bs.count().to_pyarrow().as_py():,}"

In [ ]:
data_glob = os.path.join(DATA_DIR, RAW_DATA_DIR, SOCIAL_MEDIA_TABLE, "*.parquet")
sm = ibis.read_parquet(data_glob)
sm

In [ ]:
sm = catalog.table("social_media")
sm